In [3]:
#%%
import os
import shutil
import random
from site import execsitecustomize
import numpy as np
import pandas as pd
import sys

from datetime import datetime

import math

import matplotlib.pyplot as plt
import csv
import re

import seaborn as sns






# def approxIndexRow(df, approxIndex):
#     return abs(df.index.to_series()-approxIndex).idxmin()

# Video Rendering

In [9]:
import imageio
import matplotlib.cm as cm
import matplotlib.animation as animation

## function for Video rendering:
def makeVid(imgList, outpath, fps = 2):
    frames = [] # for storing the generated images
    fig = plt.figure()
    for i in range(len(imgList)):
        frames.append([plt.imshow(imgList[i], cmap=cm.Greys_r,animated=True)])

    anim = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                    repeat_delay=1000)
    

    f = os.path.normpath(f'{outpath}.mp4')
    writergif = animation.PillowWriter(fps=fps) 
    writervideo = animation.FFMpegWriter(fps=fps) 
    # anim.save(f, writer=writergif)
    # anim.save(f)
    anim.save(f, writer=writervideo)
    plt.close() 
    return


## get directories in img folder:
imgFolder = './img/'
dirList = os.listdir(imgFolder)
folders = [f for f in dirList if os.path.isdir(os.path.join(imgFolder,f))]
imgDirList = [f for f in folders if f.startswith('T')]
imgDirList.sort()
print(imgDirList)

['T0.7_d18_x128_forcedWave_surfaceHeatmap', 'T0.7_d18_x128_pointPerturbation_surfaceHeatmap']


In [10]:
imgDirList = ['T0.7_d18_x128_forcedWave_surfaceHeatmap']

fps = 8

vidFolder = os.path.normpath('./video/')
if not os.path.exists(vidFolder): os.mkdir(vidFolder)

imgList = []
for imgDir in imgDirList:
    imgDirPath = os.path.join(imgFolder, imgDir)
    imgFiles = os.listdir(imgDirPath)
    imgFiles.sort()
    for imgFile in imgFiles:
        imgFilePath = os.path.join(imgDirPath, imgFile)
        imgList.append(imageio.imread(imgFilePath))
    vidOutPath = os.path.join(vidFolder, f'{imgDir}')
    makeVid(imgList, vidOutPath, fps)
    print(f'video for {imgDir} done.')

/tmp/ipykernel_13093/807747938.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgList.append(imageio.imread(imgFilePath))


# Image Generation

In [7]:
dirList = os.listdir('.')
folders = [f for f in dirList if os.path.isdir(f)]
print(folders)

['T0.7_d18_x128_forcedWave', 'T0.7_d18_x128_pointPerturbation', 'video', 'img']


In [8]:
folders = ['T0.7_d18_x128_forcedWave']

if not os.path.exists('./img/'): os.mkdir('./img/')

for folder in folders:
    imgFolder = os.path.join('./img/', f'{folder}_surfaceHeatmap')
    if not os.path.exists(imgFolder): os.mkdir(imgFolder)
    print(f'folder: {folder}')
    
    dirList = os.listdir(folder)
    scalarSamples = [f for f in dirList if 'CapiSampling_TS' in f]
    scalarSamples.sort()
    # print(dirList)
    if len(scalarSamples) == 0:
        print(f'no relevant files found in folder {folder}.')
    else:
        print(f"folder: {folder}")
        for sample in scalarSamples:
            print(f'sample: {sample}')
            filePath = os.path.join(folder, sample)
            sampledf = pd.read_csv(filePath, delimiter='\s+', index_col='x')

            leftDF = sampledf[sampledf['yIndex']==0]
            leftDF = leftDF.drop(['yIndex'], axis=1)

            rightDF = sampledf[sampledf['yIndex']==1]
            rightDF = rightDF.drop(['yIndex'], axis=1)

            plt.rcParams['figure.figsize'] = [12, 6]
            plt.subplot(1, 2, 1) # row 1, col 2 index 1
            ax = sns.heatmap(leftDF,annot=True, square=True, vmin=0, vmax=.5)
            plt.subplot(1, 2, 2) # row 1, col 2 index 2
            ax = sns.heatmap(rightDF,annot=True, square=True, vmin=0, vmax=.5) #, vmin=0, vmax=.8
            plt.savefig(os.path.join(imgFolder, f'{folder}_{sample}.jpg'))
            plt.close()
    

folder: T0.7_d18_x128_forcedWave
folder: T0.7_d18_x128_forcedWave
sample: CapiSampling_TS000000000.dat
sample: CapiSampling_TS000000200.dat
sample: CapiSampling_TS000000400.dat
sample: CapiSampling_TS000000600.dat
sample: CapiSampling_TS000000800.dat
sample: CapiSampling_TS000001000.dat
sample: CapiSampling_TS000001200.dat
sample: CapiSampling_TS000001400.dat
sample: CapiSampling_TS000001600.dat
sample: CapiSampling_TS000001800.dat
sample: CapiSampling_TS000002000.dat
sample: CapiSampling_TS000002200.dat
sample: CapiSampling_TS000002400.dat
sample: CapiSampling_TS000002600.dat
sample: CapiSampling_TS000002800.dat
sample: CapiSampling_TS000003000.dat
sample: CapiSampling_TS000003200.dat
sample: CapiSampling_TS000003400.dat
sample: CapiSampling_TS000003600.dat
sample: CapiSampling_TS000003800.dat
sample: CapiSampling_TS000004000.dat
sample: CapiSampling_TS000004200.dat
sample: CapiSampling_TS000004400.dat
sample: CapiSampling_TS000004600.dat
sample: CapiSampling_TS000004800.dat
sample: C

# old stuff:

In [15]:
# folders = ["./T0.7_d20"]
dirList = os.listdir('.')
folders = [f for f in dirList if os.path.isdir(f)]

for folder in folders:
    dirList = os.listdir(folder)
    scalarSamples = [f for f in dirList if 'scalquant_all_reg1' in f]
    # print(dirList)
    if len(scalarSamples) == 0:
        print(f'no relevant files found in folder {folder}.')
    else:
        print(f"folder: {folder}")
        for sample in scalarSamples:
        # sample = scalarSamples[-1]
            samplePath = os.path.join(folder, sample)
            sampledf = pd.read_csv(samplePath, delimiter='\s+')

            sampledf["rho[0]"].plot()



no relevant files found in folder T0.7_d18_x128_forcedWave.


In [22]:

def vle_kedia2006(T):
    '''
    Get saturated densities of DROPLET by Vrabec et al., Molecular Physics 104 (2006). Equation numbers refer this paper.
    :param float T: Temperature
    :return: float rhol, float rhov: Saturated liquid and vapor density
    '''
    Tc = 1.0779
    rc = 0.3190

    dT = (Tc-T)

    a,b,c=0.5649,0.1314,0.0413
    rhol=rc+a*dT**(1/3.)+b*dT+c*dT**(3/2.)       # equation 4
    a,b,c=0.5649,0.2128,0.0702
    rhov=rc-a*dT**(1/3.)+b*dT+c*dT**(3/2.)       # equation 5

    return rhol,rhov



In [23]:
vle_kedia2006(.7)

(0.7866611943875055, 0.007314397513485199)